In [1]:
import psaw
import praw
import datetime as dt
import pandas as pd
import numpy as np

import pickle
import os
from os import listdir
from os.path import isfile, join

In [2]:
from psaw import PushshiftAPI

api = PushshiftAPI()

In [3]:
today = dt.datetime.today()

today_month = today.month
today_year = today.year

start_year = 2020
start_month = 12
dates = []
while start_year <= today_year:
    
    if start_year == today_year and start_month > today_month:
        break
        
    dates.append((start_year, start_month))
    
    start_month += 1
    if start_month > 12:
        start_year += 1
        start_month = 1
    

In [4]:
subreddits = ["CrohnsDisease","UlcerativeColitis","IBD","ibs"]

In [5]:
def retrieve_ids_in_range(year,month,SR,verbose=True):
    print("Scraping","r/"+SR,"for:",year,"-",month)
    
    DAY = 60*60*24 # seconds in a day
    HOUR = 60*60
    
    # dates are timestamps in seconds = int(dt.datetime(2020, 1, 1).timestamp())
    # 1hour offset for some reason? IGNORED
    
    start=int(dt.datetime(year, month, 1).timestamp())
    if month < 12:
        end=int(dt.datetime(year, month+1, 1).timestamp())
    else: # if dec 2019
        end=int(dt.datetime(2020, 1, 1).timestamp())
    
    start_epoch = start
    end_epoch = start + DAY 
    # search IDs on a weekly basis
    ids = []
    while end_epoch <= end:
        # add ids to list
        #print(".")
        res = list(api.search_submissions(
                            after =start_epoch,
                            before=end_epoch + HOUR, 
                            subreddit=SR,
                            #filter=['url','author', 'title', 'subreddit'],
                            limit=100))
        # debug
        if verbose:
            print("FROM:",np.intc(start_epoch).astype("datetime64[s]"),"TO:",np.intc(end_epoch).astype("datetime64[s]"),)        
            print("FIRST:",np.intc(res[-1].created_utc).astype("datetime64[s]"),"LAST:",np.intc(res[0].created_utc).astype("datetime64[s]"),)        
            print("number of posts:", len(res))
        
        #append results
        for r in res:
            ids.append(r.id)
        
        #update epochs
        start_epoch = end_epoch
        end_epoch = start_epoch + DAY #60*60*24
    
    #save csv for that subreddit
    print("saving...",SR+"_"+str(year)+"-"+str(month)+"_submissions.csv")
    print(len(ids),"posts")
    pd.DataFrame(ids).drop_duplicates().to_csv("./submissions/"+SR+"_"+str(year)+"-"+str(month)+"_submissions.csv")
    return ids

In [6]:
for yy, mm in dates:
    retrieve_ids_in_range(year=yy, month=mm, SR="CrohnsDisease",verbose=False)

print("DONE")

Scraping r/CrohnsDisease for: 2020 - 12
saving... CrohnsDisease_2020-12_submissions.csv
0 posts
Scraping r/CrohnsDisease for: 2021 - 1


C:\Users\mic\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 521
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\mic\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


Exception: Unable to connect to pushshift.io. Max retries exceeded.

In [ ]:
mypath = "./submissions/"
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

x = [pd.read_csv(path,index_col=0) for path in onlyfiles]
if len(x) > 0:
    x = pd.concat(x)
    x.columns= ["id"]
    x = x.reset_index()
    x.to_csv("all_ids.csv")

In [ ]:
new_posts = pd.read_csv('./all_ids.csv',index_col=0)
new_posts.columns = ['id','subreddit']
print('new posts size:',new_posts.shape, new_posts.drop_duplicates(subset='id').shape)
new_posts = new_posts.drop_duplicates().reset_index(drop=True)

old_posts = pd.read_pickle("./all_posts.pickle")
print('old posts size:',old_posts.shape, old_posts.drop_duplicates().shape)

posts_to_scrape = new_posts[~new_posts['id'].isin(old_posts['id'])]
print('posts_to_scrapesize:',posts_to_scrape.shape)
posts_to_scrape = posts_to_scrape.reset_index(drop=True)